In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import cross_val_score, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import manifold, decomposition, linear_model, ensemble, neighbors, cross_validation, metrics


import xgboost
from xgboost import DMatrix

/Users/Roman/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [45]:
train = pd.read_csv("../datasets/train_without_oversampling_sigmoid_shk.csv")
test = pd.read_csv("../datasets/test_without_oversampling_shk.csv")

In [46]:
y = train.target
train.drop("target", axis=1, inplace=True)
train.drop("client_id", axis=1, inplace=True)

ans = test[["client_id"]]
test.drop("client_id", axis=1, inplace=True)

In [47]:
Dtrain = DMatrix(data=train.to_sparse(), label=y)

In [48]:
seed = 0
np.random.seed(seed=seed)

In [49]:
param = {
    "max_depth" : 10,
    "eta" : 0.03,
    "objective": "binary:logistic",
    "eval_metric": "auc",
    "max_delta_step": 2,
    "alpha": 2,
    "lambda": 2,
    "colsample_bytree": 0.8,
}

numround = 400
history = xgboost.cv(params=param, dtrain=Dtrain, num_boost_round=numround, nfold=3, stratified=True,\
                     early_stopping_rounds=25, verbose_eval=25, seed=seed)

[0]	train-auc:0.754274+0.00137568	test-auc:0.735893+0.0030782
[25]	train-auc:0.788698+0.000415323	test-auc:0.754167+0.00194882
[50]	train-auc:0.804261+0.000685741	test-auc:0.757865+0.00166833
[75]	train-auc:0.818823+0.000528663	test-auc:0.760028+0.00162164
[100]	train-auc:0.831525+0.000449956	test-auc:0.761211+0.00162835
[125]	train-auc:0.843533+0.000179635	test-auc:0.762078+0.00131463
[150]	train-auc:0.853756+0.000977566	test-auc:0.762642+0.00141715
[175]	train-auc:0.861022+0.00131443	test-auc:0.76309+0.00142443
[200]	train-auc:0.86669+0.00159413	test-auc:0.763386+0.00140179
[225]	train-auc:0.871633+0.00186638	test-auc:0.763738+0.00132419
[250]	train-auc:0.875901+0.00208631	test-auc:0.76397+0.00142113
[275]	train-auc:0.879737+0.00206034	test-auc:0.764145+0.00146395
[300]	train-auc:0.88359+0.0024612	test-auc:0.764212+0.00148428
[325]	train-auc:0.887294+0.0022311	test-auc:0.764216+0.00151981


In [50]:
history.tail(2)

,test-auc-mean,test-auc-std,train-auc-mean,train-auc-std
306,0.764242,0.001494,0.884415,0.002313
307,0.764268,0.001500,0.884698,0.002201


In [ ]:
from hyperopt import hp
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [ ]:
def objective(space):
    print(space)
    
    
    
    
    res = metrics.roc_auc_score(y_test, y_ans)
    print(res)
    return{'loss': -res, 'status': STATUS_OK}


space ={
        'n_estimators' : hp.quniform('n_estimators', 100, 600, 1),
        'eta' : hp.uniform('eta', 0.01, 0.4),
        'max_depth' : hp.quniform('max_depth', 3, 10, 1),
}


trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=75,
            trials=trials)

print(best)

In [ ]:
print test.shape

In [53]:
param = {
    "max_depth" : 9,
    "eta" : 0.05,
    "objective": "binary:logistic",
    "eval_metric": "auc",
    "max_delta_step": 2,
    "alpha": 2,
    "lambda": 2,
}

numround = 400
bst = xgboost.train(params=param, dtrain=Dtrain, num_boost_round=numround)

In [54]:
y_ans = bst.predict(DMatrix(data=test.to_sparse()))
ans['val'] = y_ans
ans.rename(columns={"client_id":"_ID_", "val":"_VAL_"}, inplace=True)
ans.to_csv("../solutions/xgboost_random_oversampling_sigmoid_shk.csv", index=False, sep=",", header=False)

/Users/Roman/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/Roman/anaconda/lib/python3.6/site-packages/pandas/core/frame.py:2834: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)
